In [23]:
import polars as pl
import duckdb
file_path = "D:/qmt_datadir/adjust_factor/"


df = duckdb.sql("""
    SELECT 
        *
    FROM 
        read_parquet('D:/qmt_datadir/adjust_factor/*.parquet')
    WHERE
        time >= '2023-01-01'
    ORDER BY
        time
""").pl()
df

time,ticker_symbol,interest,stockBonus,stockGift,allotNum,allotPrice,gugai,dr
datetime[μs],str,f64,f64,f64,f64,f64,f64,f64
2023-01-03 00:00:00,"""000001.SZ""",0.228,0.0,0.0,0.0,0.0,0.0,1.017996
2023-01-03 00:00:00,"""000002.SZ""",0.976126,0.0,0.0,0.0,0.0,0.0,1.062659
2023-01-03 00:00:00,"""000004.SZ""",0.0,0.25,0.0,0.0,0.0,1.0,1.25
2023-01-03 00:00:00,"""000006.SZ""",0.121,0.0,0.0,0.0,0.0,0.0,1.028037
2023-01-03 00:00:00,"""000007.SZ""",0.0,0.0,0.5,0.0,0.0,0.0,1.500315
…,…,…,…,…,…,…,…,…
2024-11-19 00:00:00,"""900945.SH""",0.002858,0.0,0.0,0.0,0.0,0.0,1.001851
2024-11-19 00:00:00,"""900947.SH""",0.00747,0.0,0.0,0.0,0.0,0.0,1.011538
2024-11-19 00:00:00,"""900948.SH""",0.138733,0.0,0.0,0.0,0.0,0.0,1.097945


In [ ]:
import glob
import datetime
date = datetime.date(2023, 1, 1)
queries = []
for file in glob.glob(f"{file_path}*.parquet"):
    q = pl.scan_parquet(file).with_columns(
        pl.col("time").cast(pl.Datetime),
    )
    queries.append(q)
dataframes = pl.collect_all(queries)
df = pl.concat(dataframes)

In [ ]:
import pyarrow.parquet as pq

# 读取 Parquet 文件的元数据
parquet_file = pq.ParquetFile("D:/qmt_datadir/adjust_factor/202405.parquet")

# 获取文件的元数据
metadata = parquet_file.metadata

# 查看压缩类型
for i in range(metadata.num_row_groups):
    row_group_metadata = metadata.row_group(i)
    for j in range(row_group_metadata.num_columns):
        column_chunk = row_group_metadata.column(j)
        print(f"Column: {column_chunk.path_in_schema}, Compression: {column_chunk.compression}")


Column: time, Compression: SNAPPY
Column: ticker_symbol, Compression: SNAPPY
Column: interest, Compression: SNAPPY
Column: stockBonus, Compression: SNAPPY
Column: stockGift, Compression: SNAPPY
Column: allotNum, Compression: SNAPPY
Column: allotPrice, Compression: SNAPPY
Column: gugai, Compression: SNAPPY
Column: dr, Compression: SNAPPY
None


In [34]:
df = df.with_columns(
    year_month=pl.col("time").dt.strftime("%Y%m")
)

In [57]:
year_month_list = df.select(pl.col("year_month")).unique().sort(by="year_month").to_series().to_list()

In [70]:
for i in year_month_list:
    temp = (
        df.lazy()
        .filter(pl.col("year_month") == i)
        .unique(subset=["time", "ticker_symbol"], keep="last", maintain_order=True)
        .select(pl.all().exclude("year_month"))
        .sort(["time", 'ticker_symbol'])
    ).collect()
    temp.write_parquet(f"f:/test/{i}.parquet", compression="snappy")

In [71]:
temp

time,ticker_symbol,interest,stockBonus,stockGift,allotNum,allotPrice,gugai,dr
datetime[μs],str,f64,f64,f64,f64,f64,f64,f64
2024-11-01 00:00:00,"""000001.SZ""",0.246,0.0,0.0,0.0,0.0,0.0,1.021872
2024-11-01 00:00:00,"""000002.SZ""",0.68,0.0,0.0,0.0,0.0,0.0,1.052187
2024-11-01 00:00:00,"""000004.SZ""",0.0,0.25,0.0,0.0,0.0,1.0,1.25
2024-11-01 00:00:00,"""000006.SZ""",0.094,0.0,0.0,0.0,0.0,0.0,1.020547
2024-11-01 00:00:00,"""000007.SZ""",0.0,0.0,0.5,0.0,0.0,0.0,1.500315
…,…,…,…,…,…,…,…,…
2024-11-19 00:00:00,"""900945.SH""",0.002858,0.0,0.0,0.0,0.0,0.0,1.001851
2024-11-19 00:00:00,"""900947.SH""",0.00747,0.0,0.0,0.0,0.0,0.0,1.011538
2024-11-19 00:00:00,"""900948.SH""",0.138733,0.0,0.0,0.0,0.0,0.0,1.097945
